In [1]:
import numpy as np
import math
import copy

In [2]:
rng = np.random.default_rng()

In [27]:
import numpy as np
import math

rng = np.random.default_rng()



class AntColony:
    def __init__(self, number_of_ants=None, alpha=1, beta=1,
                 starting_pheromone=1, Q=1, ro=0.9):
        if number_of_ants is not None and number_of_ants <= 0:
            raise Exception("Negative number of ants")
        if number_of_ants is not None and number_of_ants != math.floor(number_of_ants):
            raise Exception("Number of ants is not an intefer")
        if alpha < 0:
            raise Exception("Alpha parameter has to be positive")
        if beta < 0:
            raise Exception("Beta parameter has to be positive")
        if starting_pheromone < 0:
            raise Exception("starting_pheromone parameter has to be positive")
        if Q < 0:
            raise Exception("Q parameter has to be positive")
        if ro < 0 or ro > 1:
            raise Exception("ro parameter has to be in [0,1] range")

        self.number_of_ants = number_of_ants
        self.alpha = alpha
        self.beta = beta
        self.starting_pheromone = starting_pheromone
        self.Q = Q
        self.ro = ro

    def set_problem(self, coordinates, request, capacity, s_max, number_of_cars):
        if coordinates.shape[1] != 2:
            raise Exception("coordinates are not in IR^2")
        if request[0] != 0:
            print("Adding 0 as technical request for warehouse")
            request = np.insert(request, 0, 0)
        if coordinates.shape[0] != request.shape[0]:
            raise Exception("Different number of coordinates and requests")
        if np.sum(request <= 0) > 1:
            raise Exception("There is non positive request, other than the technical one at the warehouse")
        if np.any(request > capacity):
            raise Exception("There is request bigger than capacity. This problem is unsolvable!")
        if number_of_cars < 0:
            raise Exception("number_of_cars parameter has to be positive")

        self.problem_size = coordinates.shape[0]
        self.coordinates = coordinates
        self.request = request
        self.capacity = capacity

        self.restart()

        if self.number_of_ants is None:
            self.number_of_ants = 2 * self.problem_size

        if np.any(self.distance_matrix[0] * 2 > s_max):
            raise Exception(
                "There is distance bigger than max_s / 2. This problem is unsolvable! Check antColony.distance_matrix")

        self.s_max = s_max
        self.number_of_cars = number_of_cars

    def restart(self):
        self.pheromone_restart()
        self.calculate_distance_matrix()
        self.calculate_transition_matrix()

        self.best_cost = np.inf
        self.best_path = None
        self.best_cycles_number = None

    def calculate_distance_matrix(self):
        self.distance_matrix = np.zeros((self.problem_size, self.problem_size))

        for i in range(0, self.problem_size):
            for j in range(i + 1, self.problem_size):  # only above the diagonal
                d = np.linalg.norm(self.coordinates[i, :] - self.coordinates[j, :])
                self.distance_matrix[i, j] = d
                self.distance_matrix[j, i] = d

    def calculate_transition_matrix(self):
        # self.T_P is non standardize Probability
        modified_distance_matrix = self.distance_matrix + np.eye(self.problem_size)  # diagonal will later be set to 0
        self.T_P = self.pheromone_matrix ** self.alpha * (1 / modified_distance_matrix) ** self.beta
        np.fill_diagonal(self.T_P, 0)

    def pheromone_restart(self):
        self.pheromone_matrix = self.starting_pheromone * (np.ones((self.problem_size,
                                                                    self.problem_size)) - np.eye(self.problem_size))

    def ant_find_path(self):  # return (list of nodes, number of times to start from warehouse - number of 0 in list of nodes minus 1 - number of ant_find_circle() calls)
        visited_nodes = [0]  # start from warehouse
        number_of_cicles = 0
        while len(visited_nodes) - number_of_cicles < self.problem_size:  # in visited_nodes should be [0,1,...,self.problem_size] and additionally number_of_cicles zeros
            number_of_cicles += 1
            visited_nodes = self.ant_find_circle(visited_nodes)

        return visited_nodes, number_of_cicles

    def ant_find_circle(self, visited_nodes):
        if visited_nodes is None or len(visited_nodes) == 0 or visited_nodes[-1] != 0:
            raise Exception("ant is not at the warehouse")

        node = 0
        dist_from_warehouse = 0
        capacity_left = self.capacity

        # do_while loop
        move = self.ant_make_move(visited_nodes, capacity_left, dist_from_warehouse)
        visited_nodes.append(move)
        dist_from_warehouse += self.distance_matrix[node, move]
        capacity_left -= self.request[move]
        node = move
        while (node != 0):
            move = self.ant_make_move(visited_nodes, capacity_left, dist_from_warehouse)
            visited_nodes.append(move)
            dist_from_warehouse += self.distance_matrix[node, move]
            capacity_left -= self.request[move]
            node = move

        return visited_nodes

    def ant_make_move(self, visited_nodes, capacity_left,
                      dist_from_warehouse):  # wylosuj z rozkladu self.T_P[dopuszczalne]
        node = visited_nodes[-1]
        if node == 0 and dist_from_warehouse != 0:
            raise Exception("ant is in warehouse, but the dist_from_warehouse is not 0")
        if capacity_left < 0:
            raise Exception("There is less than none capacity left")

        possible_nodes = self.get_possible_nodes(visited_nodes, capacity_left, dist_from_warehouse)

        if not np.any(possible_nodes):
            raise Exception("There is nowhere to go :(")

        my_cum_sum = np.cumsum(self.T_P[node, possible_nodes])
        c = my_cum_sum[-1]

        u = self.uniform_drawn[self.now_iter, len(visited_nodes)] * c

        return np.where(possible_nodes)[0][np.where(u < my_cum_sum)[0][0]]  # EXP: np.where(u < my_cum_sum) == ([3,4,5],) # so the 3 was drawn

    def get_possible_nodes(self, visited_nodes, capacity_left, dist_from_warehouse):
        node = visited_nodes[-1]

        possible_nodes = np.ones(self.problem_size).astype(bool)  # (True, True, ..., True)

        possible_nodes[visited_nodes] = False

        if visited_nodes[-1] != 0:
            possible_nodes[0] = True

        possible_nodes[capacity_left < self.request] = False

        dist_to_others = self.distance_matrix[node]
        dist_to_warehouse = self.distance_matrix[0]
        possible_nodes[dist_from_warehouse + dist_to_others + dist_to_warehouse > self.s_max] = False

        return possible_nodes

    def calculate_cost(self, path):
        cost_sum = 0
        for i in range(len(path) - 1):
            cost_sum += self.distance_matrix[path[i], path[i + 1]]
        return cost_sum

    def pheromone_modify(self, paths):
        self.pheromone_matrix *= (1 - self.ro)
        delta_pheromone_matrix = np.zeros_like(self.pheromone_matrix)
        for path in paths:
            # path = [0,4,3,0,1,5,2,0]
            for i in range(len(path) - 1):
                delta_pheromone_matrix[path[i], path[i + 1]] += 1 / self.distance_matrix[path[i], path[i + 1]]
        self.pheromone_matrix += self.Q * delta_pheromone_matrix

    def single_iteration(self, print_progress, iteration):
        paths = [None for i in range(self.number_of_ants)]
        num_of_cycles = [None for i in range(self.number_of_ants)]
        for i in range(self.number_of_ants):
            paths[i], num_of_cycles[i] = self.ant_find_path()
        costs = np.array([self.calculate_cost(paths[i]) for i in range(self.number_of_ants)])

        i_min_cost = np.argmin(costs)
        if costs[i_min_cost] < self.best_cost:
            self.best_path = paths[i_min_cost]
            self.best_cost = costs[i_min_cost]
            self.best_cycles_number = num_of_cycles[i_min_cost]
            if print_progress:
                print("New best solution in {} iteration: cost = {:.2f} and uses {} trucks".format(iteration,
                                                                                                   self.best_cost,
                                                                                                   self.best_cycles_number))

        self.pheromone_modify(paths)

    def optimize(self, max_iter, print_progress=False, rng_seed=None, new=True):
        self.rng = np.random.default_rng(rng_seed)
        if new:
            self.restart()

        self.uniform_drawn = self.rng.uniform(size=(max_iter, 2 * self.problem_size))

        for i in range(max_iter):
            self.now_iter = i
            self.single_iteration(print_progress, i)

        # TODO - Czy udało się znaleść rozwiązanie z dobrą liczbą samochodów?

In [28]:
antColony = AntColony(number_of_ants=None, alpha=1, beta=1, starting_pheromone = 1, Q = 1)

In [29]:
request = np.array([10, 10, 10, 10])

In [30]:
coordinates = np.array([[365.4, 689.34], [365.3, 689.5], [365.3, 689.3], [365.2, 689.2], [365.1, 689.1]])

In [31]:
antColony.set_problem(coordinates, request, 100, 1.2, 5)

Adding 0 as technical request for warehouse


In [19]:
antColony.optimize(300, print_progress=True)

New best solution in 0 iteration: cost = 1.06 and uses 1 trucks
New best solution in 1 iteration: cost = 1.04 and uses 1 trucks
New best solution in 14 iteration: cost = 1.03 and uses 1 trucks


In [20]:
antColony.best_path

[0, 2, 3, 4, 1, 0]